In [ ]:
!wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
from dolfin import *

In [2]:
L = 25.
H = 1.
Nx = 250
Ny = 10
mesh = RectangleMesh(Point(0., 0.), Point(L, H), Nx, Ny, "crossed")

In [3]:
def eps(v):
    return sym(grad(v))

In [4]:
E = Constant(1e5)
nu = Constant(0.3)
model = "plane_stress"

In [5]:
mu = E/2/(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)
if model == "plane_stress":
    lmbda = 2*mu*lmbda/(lmbda+2*mu)

In [6]:
def sigma(v):
    return lmbda*tr(eps(v))*Identity(2) + 2.0*mu*eps(v)

In [7]:
rho_g = 1e-3
f = Constant((0, -rho_g))

In [8]:
V = VectorFunctionSpace(mesh, 'Lagrange', degree=2)
du = TrialFunction(V)
u_ = TestFunction(V)
a = inner(sigma(du), eps(u_))*dx
l = inner(f, u_)*dx

In [9]:
def left(x, on_boundary):
    return near(x[0], 0.)

In [10]:
bc = DirichletBC(V, Constant((0.,0.)), left)

In [11]:
u = Function(V, name="Displacement")
solve(a == l, u, bc)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [12]:
plot(1e3*u, mode="displacement")

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [13]:
print("Maximal deflection:", -u(L,H/2.)[1])
print("Beam theory deflection:", float(3*rho_g*L**4/2/E/H**3))

Maximal deflection: 0.005863752585578199
Beam theory deflection: 0.005859375


In [14]:
Vsig = TensorFunctionSpace(mesh, "DG", degree=0)
sig = Function(Vsig, name="Stress")
sig.assign(project(sigma(u), Vsig))
print("Stress at (0,H):", sig(0, H))

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Stress at (0,H): [ 1.72741309 -0.22361947 -0.22361947  0.40196748]


In [15]:
file_results = XDMFFile("elasticity_results.xdmf")
file_results.parameters["flush_output"] = True
file_results.parameters["functions_share_mesh"] = True
file_results.write(u, 0.)
file_results.write(sig, 0.)